<a href="https://colab.research.google.com/github/safaeOulaja/Embedding-Search-NTCIR-CLEF-TREC/blob/main/Embedding%2BSearch_NTCIR_18_DATASET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NTCIR-18 Automatic Evaluation of LLMs (AEOLLM) Task :

* automatic evaluation of answers generated by large language models (LLMs) for various tasks
1.   It contains data for evaluating LLMs on four main tasks: Summary
Generation (SG), Non-Factoid QA (NFQA), Text Expansion (TE), and Dialogue Generation (DG).
2. Each task dataset includes a set of questions and answers generated by LLMs, with human annotations providing scores for these responses.
3. The dataset is divided into training, test, and reserved sets for participants to design, test, and evaluate their methods.



In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
# Imports
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from IPython.display import display
from sklearn.metrics import precision_score, recall_score, f1_score

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
# Load the dataset from Hugging Face
from datasets import load_dataset

# Load the NTCIR-18 AEOLLM dataset
dataset = load_dataset("THUIR/AEOLLM", split='train')  # or 'test'

# Convert to a DataFrame
df = pd.DataFrame(dataset)

df.head(30)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/668 [00:00<?, ?B/s]

train_NFCATS.csv:   0%|          | 0.00/179k [00:00<?, ?B/s]

train_Xsum.csv:   0%|          | 0.00/246k [00:00<?, ?B/s]

train_dialog.csv:   0%|          | 0.00/75.0k [00:00<?, ?B/s]

...

test_dialog.csv:   0%|          | 0.00/87.0k [00:00<?, ?B/s]

test_story.csv:   0%|          | 0.00/269k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/560 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/560 [00:00<?, ? examples/s]

,taskId,taskName,questionId,question,answerId,answer,score,rank
0,3,NFCATS,0,Are constructed languages useful and do we nee...,0,"Constructed languages, also known as conlangs,...",5,1
1,3,NFCATS,0,Are constructed languages useful and do we nee...,1,Here is a 187-word answer to the question:\n\...,4,3
2,3,NFCATS,0,Are constructed languages useful and do we nee...,2,"Constructed languages, also known as artificia...",4,3
3,3,NFCATS,0,Are constructed languages useful and do we nee...,3,"Constructed languages, such as English and Fre...",4,3
4,3,NFCATS,0,Are constructed languages useful and do we nee...,4,Constructed languages are a group of artificia...,4,3
5,3,NFCATS,0,Are constructed languages useful and do we nee...,5,Constructed languages are an invaluable tool f...,4,3
6,3,NFCATS,0,Are constructed languages useful and do we nee...,6,"Constructed languages, also known as artificia...",5,1
7,3,NFCATS,1,Should Anyone Be Allowed To Be a Parent?,0,The question of whether anyone should be allow...,5,1
8,3,NFCATS,1,Should Anyone Be Allowed To Be a Parent?,1,"Here is a 187-word answer to the question ""Sh...",4,2
9,3,NFCATS,1,Should Anyone Be Allowed To Be a Parent?,2,The ability to raise a child is a fundamental ...,3,5


In [ ]:
# Combine question and answer text into a single 'Combined_Text' column
df['Combined_Text'] = df['question'] + " " + df['answer']

# Initialize the model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Generate embeddings for each Combined_Text
df['Embedding_Vector'] = df['Combined_Text'].apply(lambda x: model.encode(x))

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Define the search function
def search_question_answer(model, df, query, n=3):
    # Generate the embedding vector for the query
    query_embedding = model.encode([query])

    # Calculate cosine similarity between the query embedding and each embedding in 'Embedding_Vector'
    df['Cosine_Similarity'] = df['Embedding_Vector'].apply(
        lambda x: cosine_similarity([query_embedding[0]], [x])[0][0]
    )

    # Sort by cosine similarity in descending order and return the top 'n' results
    top_n_df = df.sort_values(by='Cosine_Similarity', ascending=False).head(n)

    # Display results
    for index, row in top_n_df.iterrows():
        print(f"Question: {row['question']}")
        print(f"Answer: {row['answer']}")
        print(f"Cosine Similarity: {row['Cosine_Similarity']:.4f}")
        print("\n")

# Example usage of the search function
query = 'Should scientists contribute to Wikipedia'
search_question_answer(model, df, query)

Question: Should scientists contribute to Wikipedia?
Answer: Yes, scientists should contribute to Wikipedia. The platform allows users to create, edit, and share information, which can be used by scientists to share their research and collaborate with others in their field. Wikipedia has a large user base of researchers, which can help scientists to get their work out there and gain more recognition. Additionally, by contributing to Wikipedia, scientists can learn about the importance of accuracy and completeness in information, which can be useful in their own research practices.

In conclusion, scientists should contribute to Wikipedia. It is a great platform for researchers to share their work, collaborate with others, and learn about the importance of accuracy and completeness in information.
Cosine Similarity: 0.8903


Question: Should scientists contribute to Wikipedia?
Answer: Yes, scientists should contribute to Wikipedia. It is a great platform to share their knowledge with th

In [ ]:
def calculate_metrics(df, query_embedding, top_k=30, relevance_threshold=5):
    # Calculate cosine similarity
    df['Cosine_Similarity'] = df['Embedding_Vector'].apply(
        lambda x: cosine_similarity([query_embedding[0]], [x])[0][0]
    )

    # Sort by similarity
    sorted_df = df.sort_values(by='Cosine_Similarity', ascending=False)

    # Get top-k results
    top_k_results = sorted_df.head(top_k)

    # Convert scores to binary relevance
    relevant = top_k_results['score'].apply(lambda x: 1 if x >= relevance_threshold else 0).values

    # Precision@30
    precision_at_k = precision_score(relevant, np.ones_like(relevant), zero_division=0)

    # Recall@30
    recall_at_k = recall_score(relevant, np.ones_like(relevant), zero_division=0)

    # F1-score
    f1 = f1_score(relevant, np.ones_like(relevant), zero_division=0)

    # mAP
    aps = []
    relevant_count = 0
    for i, rel in enumerate(relevant, start=1):
        if rel == 1:
            relevant_count += 1
            aps.append(relevant_count / i)
    mean_ap = np.mean(aps) if aps else 0.0

    return precision_at_k, recall_at_k, f1, mean_ap

# Setting the query and the relevance threshold
query = "Are constructed languages useful and do we need them?"
relevance_threshold = 3

# Generate embedding for the query
query_embedding = model.encode([query])

# Calculate metrics
precision_at_30, recall_at_30, f1_at_30, mean_ap = calculate_metrics(df, query_embedding, top_k=30, relevance_threshold=relevance_threshold)

# Display the metrics
print(f"Precision@30: {precision_at_30:.4f}")
print(f"Recall@30: {recall_at_30:.4f}")
print(f"F1-score@30: {f1_at_30:.4f}")
print(f"mAP: {mean_ap:.4f}")

Precision@30: 0.8667
Recall@30: 1.0000
F1-score@30: 0.9286
mAP: 0.9302
